In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Type, Union, List

In [32]:
class BasicBlock(nn.Module):
    expansion: int = 1 # BasicBlock에서는 안씀
    
    def __init__(self, in_planes: int, planes: int, stride: int = 1, padding: int = 1) -> None:
        super().__init__()

        if padding > 1:
            raise NotImplementedError("Padding > 1 not supported in BasicBlock")
            
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=padding)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=padding)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        
    def forward(self, x: Tensor) -> Tensor:
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += self.shortcut(identity)
        out = self.relu(out)
        
        return out

In [3]:
class BottleNeck(nn.Module):
    expansion: int = 4 # relu와 같은 activation에서 사라지는 정보를 최대한 줄이기 위해 사용(mobilenet 논문 참고)
    
    def __init__(self, in_planes: int, planes: int, stride: int = 1, padding: int = 1) -> None:
        super().__init__()
        
        # 1x1 conv는 연산량을 줄이기 위해 사용
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=padding)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != (planes * self.expansion):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )
        
    def forward(self, x: Tensor) -> Tensor:
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        out += self.shortcut(identity)
        out = self.relu(out)
        
        return out

In [33]:
class ResNet(nn.Module):
    def __init__(self,
                 block: Type[Union[BasicBlock, BottleNeck]],
                 layers: List,
                 num_class: int = 10) -> None:
        super().__init__()

        self.in_planes = 64
        
        # RGB Channels -> 64 Channels
        # 7x7 conv 사용 이유는, 초기의 expensive한 연산을 피하기 위해서이다.
        # aggresive하게 downsampling을 진행함으로써, resolution을 빠르게 줄인다.
        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # make layers
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        
    def _make_layer(self, 
                    block: Type[Union[BasicBlock, BottleNeck]],
                    planes: int,
                    num_block: int,
                    stride: int = 1) -> nn.Sequential:            
        
        layers = []
        layers.append(block(self.in_planes, planes, stride))
        
        self.in_planes = planes * block.expansion
        
        for _ in range(num_blocks - 1):
            layers.append(block(self.in_planes, planes, stride=stride))
            
        return nn.Sequential(*layers)
        
        
    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

In [34]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def ResNet50():
    return ResNet(BottleNeck, [3, 4, 6, 3])

def ResNet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def ResNet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])